In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.table import vstack
from multiprocessing.pool import ThreadPool

from concurrent.futures import ThreadPoolExecutor

### Import Refit data

In [ ]:
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}  ## There is no SITELLE data for NGC 4254
galaxy = galdic[5]
infile = open(f"/home/habjan/jupfiles/data/{galaxy}_refit+SITELLEfits_data.fits",'rb')
data = Table.read(infile)
galaxy

### Set parameters

In [ ]:
mciters = 10**3
snerr = 0

### PyNeb Diagnostics

In [ ]:
diags = pn.Diagnostics()

diags.addDiag('[OII] b3727/b7325', ('O2', '(L(3726)+L(3729))/(L(7319)+L(7320)+L(7330)+L(7331))', 
                                    'RMS([E(3727A+),E(7319A+)*L(7319A+)/(L(7319A+)+L(7330A+)),E(7330A+)*L(7330A+)/(L(7319A+)+L(7330A+))])'))

### SIII and OIII Temperature and SII Density Uncertainty Function

In [ ]:
def siiitemperr(indata, iters):
    
    stemperr = np.zeros(iters)
    otemperr = np.zeros(iters)
    denerr = np.zeros(iters)
    
    for j in range(iters):
        noise6312 = np.random.normal(0, indata['SIII6312_FLUX_CORR_REFIT_ERR'])
        noise9069 = np.random.normal(0, indata['SIII9068_FLUX_CORR_ERR'])
        noise6730 = np.random.normal(0, indata['SII6730_FLUX_CORR_ERR'])
        noise6716 = np.random.normal(0, indata['SII6716_FLUX_CORR_ERR'])

        stemperr[j], denerr[j] = diags.getCrossTemDen(diag_tem = '[SIII] 6312/9069', diag_den = '[SII] 6731/6716', 
                                                     value_tem = (indata['SIII6312_FLUX_CORR_REFIT']+noise6312)/(indata['SIII9068_FLUX_CORR']+noise9069), 
                                                     value_den = (indata['SII6730_FLUX_CORR']+noise6730)/(indata['SII6716_FLUX_CORR']+noise6716), 
                                                     guess_tem = 10**4)
        otemperr[j] = 0.7092 * stemperr[j] + 3609.9
            
    stemperr = np.array([stemperr[i] for i in range(len(stemperr)) if np.isnan(stemperr[i]) == False])
    otemperr = np.array([otemperr[i] for i in range(len(otemperr)) if np.isnan(otemperr[i]) == False])
    denerr = np.array([denerr[i] for i in range(len(denerr)) if np.isnan(denerr[i]) == False])
    
    return np.std(stemperr), np.std(otemperr), np.std(denerr)

### OII Temperature and SII Density Uncertainty Function

In [ ]:
def oiitemperr(indata, iters):
    
    otemperr = np.zeros(iters)
    denerr = np.zeros(iters)
    
    for j in range(iters):
        noise3727 = np.random.normal(0, indata['OII3727_FLUX_CORR_ERR'])
        noise7320 = np.random.normal(0, indata['OII7319_FLUX_CORR_REFIT_ERR'])
        noise7330 = np.random.normal(0, indata['OII7330_FLUX_CORR_REFIT_ERR'])
        noise6730 = np.random.normal(0, indata['SII6730_FLUX_CORR_ERR'])
        noise6716 = np.random.normal(0, indata['SII6716_FLUX_CORR_ERR'])

        otemperr[j], denerr[j] = diags.getCrossTemDen(diag_tem = '[OII] b3727/b7325', diag_den = '[SII] 6731/6716', 
                                                        value_tem = (indata['OII3727_FLUX_CORR']+noise3727) / (indata['OII7319_FLUX_CORR_REFIT'] + noise7320 + indata['OII7330_FLUX_CORR_REFIT'] + noise7330), 
                                                        value_den = (indata['SII6730_FLUX_CORR']+noise6730)/(indata['SII6716_FLUX_CORR']+noise6716), 
                                                        guess_tem = 10**4)
            
    otemperr = np.array([otemperr[i] for i in range(len(otemperr)) if np.isnan(otemperr[i]) == False])
    denerr = np.array([denerr[i] for i in range(len(denerr)) if np.isnan(denerr[i]) == False])
    
    return np.std(otemperr), np.std(denerr)

### NII temperature and SII density uncertainty function

In [ ]:
def niitemperr(indata, iters):
    
    temperr = np.zeros(iters)
    denerr = np.zeros(iters)
    
    for j in range(iters):
        noise5754 = np.random.normal(0, indata['NII5754_FLUX_CORR_REFIT_ERR'])
        noise6583 = np.random.normal(0, indata['NII6583_FLUX_CORR_ERR'])
        noise6730 = np.random.normal(0, indata['SII6730_FLUX_CORR_ERR'])
        noise6716 = np.random.normal(0, indata['SII6716_FLUX_CORR_ERR'])

        temperr[j], denerr[j] = diags.getCrossTemDen(diag_tem = '[NII] 5755/6584', diag_den = '[SII] 6731/6716', 
                                                     value_tem = (indata['NII5754_FLUX_CORR_REFIT']+noise5754)/(indata['NII6583_FLUX_CORR']+noise6583), 
                                                     value_den = (indata['SII6730_FLUX_CORR']+noise6730)/(indata['SII6716_FLUX_CORR']+noise6716), 
                                                     guess_tem = 10**4)
            
    temperr = np.array([temperr[i] for i in range(len(temperr)) if np.isnan(temperr[i]) == False])
    denerr = np.array([denerr[i] for i in range(len(denerr)) if np.isnan(denerr[i]) == False])
    
    return np.std(temperr), np.std(denerr)

### Temperature and Denisty Function

In [ ]:
def dentemp(indata, err, iters):

  teoii = np.zeros(len(indata))
  neoii = np.zeros(len(indata))
  teoiierr = np.zeros(len(indata))
  neoiierr = np.zeros(len(indata))
  for i in range(len(indata)):
        if indata[i]['BPT_NII'] == 0 and indata[i]['OII3727_FLUX_CORR'] > err * indata[i]['OII3727_FLUX_CORR_ERR'] and indata[i]['SII6730_FLUX_CORR'] > err * indata[i]['SII6730_FLUX_CORR_ERR'] and\
            indata[i]['SII6716_FLUX_CORR'] > err * indata[i]['SII6716_FLUX_CORR_ERR'] and indata[i]['OII7319_FLUX_CORR_REFIT'] > err * indata[i]['OII7319_FLUX_CORR_REFIT_ERR'] and\
                indata[i]['OII7330_FLUX_CORR_REFIT'] > err * indata[i]['OII7330_FLUX_CORR_REFIT_ERR']:
            teoii[i], neoii[i] = diags.getCrossTemDen(diag_tem = '[OII] b3727/b7325', diag_den = '[SII] 6731/6716', 
                                                    value_tem = indata[i]['OII3727_FLUX_CORR'] / (indata[i]['OII7319_FLUX_CORR_REFIT'] + indata[i]['OII7330_FLUX_CORR_REFIT']), 
                                                    value_den = indata[i]['SII6730_FLUX_CORR']/indata[i]['SII6716_FLUX_CORR'], 
                                                    guess_tem = 10**4)
            teoiierr[i], neoiierr[i] = oiitemperr(indata[i], iters)
        else:
            teoii[i], neoii[i] = np.nan, np.nan
            teoiierr[i], neoiierr[i] = np.nan, np.nan

  tenii = np.zeros(len(indata))
  nenii = np.zeros(len(indata))
  teniierr = np.zeros(len(indata))
  neniierr = np.zeros(len(indata))
  for i in range(len(indata)):
        if indata[i]['BPT_NII'] == 0 and indata[i]['NII5754_FLUX_CORR_REFIT'] > err * indata[i]['NII5754_FLUX_CORR_REFIT_ERR'] and indata[i]['SII6730_FLUX_CORR'] > err * indata[i]['SII6730_FLUX_CORR_ERR'] and\
            indata[i]['SII6716_FLUX_CORR'] > err * indata[i]['SII6716_FLUX_CORR_ERR'] and indata[i]['NII6583_FLUX_CORR'] > err * indata[i]['NII6583_FLUX_CORR_ERR']:
            tenii[i], nenii[i] = diags.getCrossTemDen(diag_tem = '[NII] 5755/6584', diag_den = '[SII] 6731/6716', 
                                                    value_tem = indata[i]['NII5754_FLUX_CORR_REFIT']/indata[i]['NII6583_FLUX_CORR'], 
                                                    value_den = indata[i]['SII6730_FLUX_CORR']/indata[i]['SII6716_FLUX_CORR'], 
                                                    guess_tem = 10**4)
            teniierr[i], neniierr[i] = niitemperr(indata[i], iters)
        else:
            tenii[i], nenii[i] = np.nan, np.nan
            teniierr[i], neniierr[i] = np.nan, np.nan

  tesiii = np.zeros(len(indata))
  nesiii = np.zeros(len(indata))
  tesiiierr = np.zeros(len(indata))
  nesiiierr = np.zeros(len(indata))
  teoiii = np.zeros(len(indata))
  teoiiierr = np.zeros(len(indata))
  for i in range(len(indata)):
    if indata[i]['BPT_NII'] == 0 and indata[i]['SIII9068_FLUX_CORR'] > err * indata[i]['SIII9068_FLUX_CORR_ERR'] and indata[i]['SII6730_FLUX_CORR'] > err * indata[i]['SII6730_FLUX_CORR_ERR'] and\
        indata[i]['SII6716_FLUX_CORR'] > err * indata[i]['SII6716_FLUX_CORR_ERR'] and indata[i]['SIII6312_FLUX_CORR_REFIT'] > err * indata[i]['SIII6312_FLUX_CORR_REFIT_ERR']:
        tesiii[i], nesiii[i] = diags.getCrossTemDen(diag_tem = '[SIII] 6312/9069', diag_den = '[SII] 6731/6716', 
                                        value_tem = indata[i]['SIII6312_FLUX_CORR_REFIT']/indata[i]['SIII9068_FLUX_CORR'], 
                                              value_den = indata[i]['SII6730_FLUX_CORR']/indata[i]['SII6716_FLUX_CORR'], 
                                                    guess_tem = 10**4)
        tesiiierr[i], teoiiierr[i], nesiiierr[i] = siiitemperr(indata[i], iters)

        if tesiii[i] < 14000:
            teoiii[i] = 0.7092 * tesiii[i] + 3609.9
        else: 
            teoiii[i], teoiiierr[i] = np.nan, np.nan
    else: 
        tesiii[i], nesiii[i] = np.nan, np.nan
        tesiiierr[i], nesiiierr[i] = np.nan, np.nan
        teoiii[i], teoiiierr[i] = np.nan, np.nan

  indata.add_columns([teoii, tenii, tesiii, teoiii, neoii, nenii, nesiii, teoiierr, teniierr, tesiiierr, teoiiierr, neoiierr, neniierr, nesiiierr], 
                      names=('OII_TEMP', 'NII_TEMP', 'SIII_TEMP','OIII_TEMP', 'SII_DEN_OII', 'SII_DEN_NII', 'SII_DEN_SIII',
                            'OII_TEMP_ERR', 'NII_TEMP_ERR', 'SIII_TEMP_ERR','OIII_TEMP_ERR', 'SII_DEN_OII_ERR', 'SII_DEN_NII_ERR', 'SII_DEN_SIII_ERR'))

  return indata

# Multiprocessing of Temperature and Density Data

In [ ]:
if __name__ == "__main__":

    corenum = 32                            #chosen based of the number of cores
    batch = math.ceil(len(data)/corenum)     #batch determines the number of data points in each batched dataset
    datalist = [data[i:i+batch] for i in range(0, len(data), batch)] #make list of batched data
    
    pool = mp.Pool(processes = len(datalist))          #count processes are inititiated
    mplist = [pool.apply_async(dentemp, args = (d, snerr, mciters)) for d in datalist] #each batched dataset is assigned to a core 

In [ ]:
results = [mplist[i].get() for i in range(len(mplist))]      #Retrieve parallelized results

# Metallicity

### PyNeb atomic class information

In [ ]:
icf = pn.ICF()
O1 = pn.Atom('O', 1)
O2 = pn.Atom('O', 2)
O3 = pn.Atom('O', 3)
icf_list = ['Ial06_16']

### Monte Carlo Metallicity Functions

In [ ]:
def metal_err(indata, iters):
    meterr = np.zeros(iters)
    ion2err = np.zeros(iters)
    ion3err = np.zeros(iters)
    
    for j in range(iters):
        
        noise3727 = np.random.normal(0, indata['OII3727_FLUX_CORR_ERR'])
        noise7319 = np.random.normal(0, indata['OII7319_FLUX_CORR_REFIT_ERR'])
        noise7330 = np.random.normal(0, indata['OII7330_FLUX_CORR_REFIT_ERR'])
        noisetemp2 = np.random.normal(0, indata['OII_TEMP_ERR'])
        noiseden2 = np.random.normal(0, indata['SII_DEN_OII_ERR'])
        noise4861 = np.random.normal(0, indata['HB4861_FLUX_CORR_ERR'])
        noise5006 = np.random.normal(0, indata['OIII5006_FLUX_CORR_ERR'])
        noisetemp3 = np.random.normal(0, indata['OIII_TEMP_ERR'])
        noiseden3 = np.random.normal(0, indata['SII_DEN_SIII_ERR'])
        
        OII = O2.getIonAbundance(int_ratio = (indata['OII3727_FLUX_CORR'] + noise3727 + indata['OII7330_FLUX_CORR_REFIT'] + noise7330 + indata['OII7319_FLUX_CORR_REFIT'] + noise7319), 
                                 tem = indata['OII_TEMP'] + noisetemp2, 
                                 den= indata['SII_DEN_OII'] + noiseden2, 
                                 to_eval='L(3726)+L(3729)+L(7319)+L(7320)+L(7330)+L(7331)', 
                                 Hbeta = indata['HB4861_FLUX_CORR'] + noise4861)
        
        OIII = O3.getIonAbundance(int_ratio = (indata['OIII5006_FLUX_CORR']+noise5006), 
                                      tem = (indata['OIII_TEMP']+noisetemp3), 
                                      den= (indata['SII_DEN_SIII']+noiseden3), 
                                      to_eval='L(5007)', 
                                      Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
            
        abunlist = {'O2': OII, 'O3': OIII}
        abundance = icf.getElemAbundance(abunlist, icf_list)
        meterr[j] = 12 + np.log10(abundance['Ial06_16'])
        ion2err[j] = OII
        ion3err[j] = OIII
        
    meterr = np.array([meterr[i] for i in range(len(meterr)) if np.isnan(meterr[i]) == False])
    ion2err = np.array([ion2err[i] for i in range(len(ion2err)) if np.isnan(ion2err[i]) == False])
    ion3err = np.array([ion3err[i] for i in range(len(ion3err)) if np.isnan(ion3err[i]) == False])
    
    return np.std(meterr), np.std(ion2err), np.std(ion3err)

### Metllicity Function

In [ ]:
def metal(indata, err, iters):

    met = np.zeros(len(indata))
    ion2 = np.zeros(len(indata))
    ion3 = np.zeros(len(indata))
    meterr = np.zeros(len(indata))
    ion2err = np.zeros(len(indata))
    ion3err = np.zeros(len(indata))
    
    for j in range(len(indata)):
        if indata[j]['OII7330_FLUX_CORR_REFIT'] > err * indata[j]['OII7330_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['OII7319_FLUX_CORR_REFIT'] > err * indata[j]['OII7319_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['OII3727_FLUX_CORR'] > err * indata[j]['OII3727_FLUX_CORR_ERR'] and \
        indata[j]['HB4861_FLUX_CORR'] > err * indata[j]['HB4861_FLUX_CORR'] and \
        indata[j]['OIII5006_FLUX_CORR'] > err * indata[j]['OIII5006_FLUX_CORR_ERR']:
            
            OII = O2.getIonAbundance(int_ratio = (indata[j]['OII3727_FLUX_CORR'] + indata[j]['OII7330_FLUX_CORR_REFIT']+indata[j]['OII7319_FLUX_CORR_REFIT']), 
                                         tem = indata[j]['OII_TEMP'], den= indata[j]['SII_DEN_OII'], 
                                         to_eval='L(3726)+L(3729)+L(7319)+L(7320)+L(7330)+L(7331)', Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            OIII = O3.getIonAbundance(int_ratio = indata[j]['OIII5006_FLUX_CORR'], tem = indata[j]['OIII_TEMP'], 
                                      den= indata[j]['SII_DEN_SIII'], to_eval='L(5007)', 
                                      Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            abunlist = {'O2': OII, 'O3': OIII}
            abundance = icf.getElemAbundance(abunlist, icf_list)
            met[j] = 12 + np.log10(abundance['Ial06_16'])
            ion2[j] = OII
            ion3[j] = OIII
            meterr[j], ion2err[j], ion3err[j] = metal_err(indata[j], iters)
        else: 
            met[j], ion2[j], ion3[j] = np.nan, np.nan, np.nan
            meterr[j], ion2err[j], ion3err[j] = np.nan, np.nan, np.nan

    indata.add_columns([met, ion2, ion3, meterr, ion2err, ion3err],
                     names=('MET_DIRECT', 'OII_ABUN', 'OIII_ABUN', 
                     'MET_DIRECT_ERR', 'OII_ABUN_ERR', 'OIII_ABUN_ERR'))

    return indata

### Create multiprocessing objects

In [ ]:
if __name__ == "__main__":
    
    pool = mp.Pool(processes = len(results))          #count processes are inititiated
    mplist2 = [pool.apply_async(metal, args = (r, snerr, 5000)) for r in results]

### Get results from multiprocessing objects

In [ ]:
results2 = [mplist2[i].get() for i in range(len(mplist2))]
metdata = vstack(results2)

# Save Metallicity Results

In [ ]:
metdata.write(f'/home/habjan/jupfiles/data/{galaxy}_physdata_MUSE+SITELLE.fits', overwrite=True)  #, overwrite=True